In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json
import numpy as np
import pandas as pd
import glob
import scipy.stats as st

import sys
sys.path.append("./../../cuhksz-phd/sho_util/pyfiles/")
from basic import get_bool_base_on_conditions

def get_json(path):
    with open(path, 'r') as j:
         contents = json.loads(j.read())
    return contents

models = {
    "ground-truth": "ground-truth/",
    # ground truth
    "External---gtHED": "External/gtHED/",
    "VA---gtHED": "Variance-Adaptor/gtHED/",
    "VA-seq---gtHED": "Variance-Adaptor_seq/gtHED/",
    # predicted HED
    "External---predHED_spkemb_noseq": "External/predHED---External---spkemb_noseq/",
    "External---predHED_spkemb": "External/predHED---External---spkemb/",
    "VA---predHED": "Variance-Adaptor/predHED---Variance-Adaptor/",
    "VA-seq---predHED": "Variance-Adaptor_seq/predHED---Variance-Adaptor_seq/",
}

In [ ]:
def get_control_result(contents, name, sampleid, segment):
    arrays = []
    for idx, a in enumerate(contents):
        if idx>=48:
            continue
        mn, fileid, emotion = a["TestID"].split("---")[-3:]
        allocation = {item[0]: item[2] for item in a["PresentationOrder"].split(", ")}
        res = [allocation[key] for key in a["Preference"]]
        array = [mn, fileid, emotion, *res]
        arrays += [array]

    columns = ["model name", "file id", "emotion", "Lowest Prediction", "Highest Prediction"]
    sampledf = pd.DataFrame(np.array(arrays), columns=columns)
    samplearrays = []
    for mn in modelnames:
        for fileid in sampledf["file id"].unique():
            params = {"model name": [mn], "file id": [fileid]}
            modeldf = sampledf[get_bool_base_on_conditions(sampledf, params)]
            if len(modeldf)>0:
                int_array = []
                for emotion in emos:
                    params = {"emotion": [emotion]}
                    emotiondf = modeldf[get_bool_base_on_conditions(modeldf, params)]
                    a = [(emotiondf["Lowest Prediction"]==label).sum() for label in intensitylabels]
                    b = [(emotiondf["Highest Prediction"]==label).sum() for label in intensitylabels]
                    int_array += [a + b]
                array = list(np.array(int_array).reshape(-1)) + list(np.sum(np.array(int_array), axis=0))
            else:
                array = [None]*30
            samplearrays += [[name, sampleid, fileid, mn, *array]]

    columns = []
    for cl in ["Test Name", "Test ID", "File ID", "model name"]:
        columns += [("basic", "", cl)]
    for emotion in emos + ["total"]:
        for key in ["LP", "HP"]:
            for label in intensitylabels:
                columns += [(f"{segment}-level control", emotion, f"{key}-{label}")]
    result = pd.DataFrame(np.array(samplearrays), columns=pd.MultiIndex.from_tuples(columns))
    return result

def get_mushra_result(contents, name, sampleid, metric):
    arrays = []
    for idx, a in enumerate(contents):
        if idx>=100:
            continue
        if len(a)==1:
            continue
        _, fileid = a["TestID"].split("---")
        ratings = a["rating"]
        arrays += [[name, sampleid, fileid, key, ratings[key]]for key in ratings]
    columns = []
    for cl in ["Test Name", "Test ID", "File ID", "model name"]:
        columns += [("basic",  cl)]
    for label in [f"{metric}"]:
        columns += [(metric, "score")]
    result = pd.DataFrame(np.array(arrays), columns=pd.MultiIndex.from_tuples(columns))
    result.loc[:, [metric]] = result.loc[:, [metric]].values.astype(float)
    return result

def get_contents(path):
    contents = get_json(path)
    name = contents[-1]["UserName"]
    sampleid = path[:-5].split("_")[-1]
    return contents, name, sampleid

In [ ]:
resultfiles = glob.glob(f"./web_service/results/*.json")
resultfiles.sort()

In [ ]:
not_found = []
results = {key: [] for key in ["SIM", "NAT"]}
for path in resultfiles:
    contents, name, sampleid = get_contents(path)
    if len(contents)==2:
        continue
    testtype = contents[0]["TestID"].split("---")[0]
    result = get_mushra_result(contents, name, sampleid, testtype)
    results[testtype] += [result]

# Check Individually

In [ ]:
idx = 1
metric = "NAT"

result = results[metric][idx]
print(result[("basic", "Test Name")][0])
df = result.groupby([("basic","Test Name"), ("basic","model name")]).mean()[metric]
df

# Summary

In [ ]:
def color_gradient(val, min_val=-1, max_val=1, color='red'):
    if val < min_val: val = min_val
    if val > max_val: val = max_val
    normalized_val = (val - min_val) / (max_val - min_val)
    
    if color == 'red':
        red_intensity = int(255 * normalized_val)
        return f'background-color: rgb({red_intensity}, 0, 0)'
    elif color == 'green':
        green_intensity = int(255 * normalized_val)
        return f'background-color: rgb(0, {green_intensity}, 0)'
    elif color == 'blue':
        blue_intensity = int(255 * normalized_val)
        return f'background-color: rgb(0, 0, {blue_intensity})'
    else:
        return 'background-color: none'

def apply_color_gradient(df, min_val=0, max_val=1.0):
    """
    df.style.apply(lambda x:apply_color_gradient(x, min_val=0, max_val=100), axis=None)
    """
    styles = df.copy()
    for col in styles.columns:
        color = 'red' if "LP" in col else "green"
        color = "blue" if col=="mean" else color
        styles[col] = df[col].apply(color_gradient, min_val=min_val, max_val=max_val, color=color)
    return styles

- MUSHRA

In [ ]:
standardization = False
excluded_list = []

modelnames = list(models.keys())
modelnames[0] = modelnames[0].replace("ground-truth", "ground_truth")
dfdir = {}
for metric in ["NAT", "SIM"]:
    mns = modelnames[1:] if metric=="SIM" else modelnames
    df_list = []
        
    df = pd.concat([results[metric][idx] for idx in range(len(results[metric]))], axis=0)
    params = {("basic", "Test Name"): list(set(list(df[("basic", "Test Name")].unique())) - set(excluded_list))}
    df = df[get_bool_base_on_conditions(df, params, True)]
    if standardization:
        for testid in df[("basic", "Test ID")].unique():
            a = df[get_bool_base_on_conditions(df, {("basic", "Test ID"): [testid]}, True)]
            col = (metric, "score")
            df.loc[get_bool_base_on_conditions(df, {("basic", "Test ID"): [testid]}, True), col]  = (a[col] - a[col].mean()) / a[col].std() 
    arrays = []
    for mn in mns:
        params = {("basic", "model name"): [mn]}
        d = df[get_bool_base_on_conditions(df, params, True)]
        values = d[(metric, "score")].values
        mean = np.mean(values)
        ivl = st.t.interval(confidence=0.95, df=len(values)-1, loc=np.mean(values), scale=st.sem(values)) 
        ivl = (ivl[1]-ivl[0])/2
        arrays += [[mean, ivl]]
    df = pd.DataFrame(arrays, index=mns, columns=["mean", "interval"])
    dfdir[metric] = df

In [ ]:
df = dfdir["NAT"]
df.style.apply(lambda x:apply_color_gradient(x, min_val=df["mean"].min(), max_val=df["mean"].max()), axis=None, subset=["mean"])

In [ ]:
df = dfdir["SIM"]
df.style.apply(lambda x:apply_color_gradient(x, min_val=df["mean"].min(), max_val=df["mean"].max()), axis=None, subset=["mean"])